# Predict The Power - Version 2.1

**v1.0**
---
I am going to use Linear Regression with almost no feature engineering, only fill the the missing values. *(scored: 42)*

**v2.0**
---
I have learned a basic technique to use deep neural network for tabular data. I am going apply that in this problem and see what happens. *(scored: 68)*

**v2.0.1**
---
Instead of using the Standard Scaler on the whole dataset (numerical features), I am going to use it separately on every numerical feature. *(yeilded the same scaled vector as v2.0, silly me. I need to learn more about data science.)*

**v2.1**
---
Let's see what happens without any scaling.

[Competition Link](https://www.hackerearth.com/challenges/competitive/hackerearth-machine-learning-challenge-predict-windmill-power/)

In [1]:
%cd '/content/drive/Shareddrives/With Myself/HackerEarth-ml-competitions/predict-the-power'

/content/drive/Shareddrives/With Myself/HackerEarth-ml-competitions/predict-the-power


My academic email comes with unlimited google drive storage. I have created a shared drive there and shared it with my other google accounts, so that I can store the huge datasets for ML and DL.

## Imports

In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Reshape, Concatenate, Dropout, BatchNormalization
from tensorflow.keras import Model

## Data

In [3]:
!ls

'predictions (#1).csv'	 sample_submission.csv	 train.csv
'predictions (#2).csv'	 test.csv


### Load the data

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### Take a peek into the data

In [5]:
train.head()

,tracking_id,datetime,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
0,WM_33725,2019-08-04 14:33:20,94.820023,-99.000000,41.723019,-0.903423,82.410573,42.523015,2563.124522,76.665560,103402.961872,26.897875,NaN,239.836388,2730.310605,42.084666,BA,Medium,2.217542,0.314065,24.281689,6.766521
1,WM_698,2018-11-05 10:13:20,241.832734,27.764785,-99.000000,-99.000000,44.104919,46.258870,2372.384119,78.129803,17030.904078,39.801469,NaN,337.944723,1780.207200,107.888643,A2,Medium,4.210346,0.448494,27.262139,5.966275
2,WM_39146,2019-09-14 14:03:20,95.484724,NaN,41.855473,12.652763,42.322098,42.878552,1657.169646,67.654469,16125.927107,36.116065,45.033197,227.850294,1666.049900,-42.931459,ABC,Medium,2.719475,0.302321,27.366127,2.874342
3,WM_6757,2018-12-25 15:33:20,238.819424,-99.000000,45.443914,15.115323,44.759643,47.282101,2888.134079,95.389974,18689.732336,46.020045,44.827154,492.081520,1964.502895,42.744596,ABC,NaN,4.857385,0.367140,24.287767,14.851089
4,WM_21521,2019-05-04 03:13:20,10.722890,NaN,41.981183,1.715696,-17.616459,43.469852,781.695419,37.423065,114468.169007,34.572941,-99.000000,259.274601,1177.516152,13.387289,AAA,Medium,NaN,0.453374,27.971650,3.519074


In [6]:
test.head()

,tracking_id,datetime,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m)
0,WM_19817,2019-04-17 08:53:20,94.324266,17.641186,89.714193,51.146788,40.460560,39.594734,1073.202715,66.830037,16681.044117,24.004812,43.756693,445.976992,1664.222023,21.912243,BA,Medium,3.185837,0.403965,25.572431
1,WM_18723,2019-03-30 07:43:20,10.088870,13.978119,43.272846,46.516394,40.027788,41.176860,517.436430,37.284163,NaN,29.431813,42.728174,499.595287,1165.111992,-35.050093,A,Medium,3.016603,0.444755,24.371823
2,WM_34552,2019-08-10 11:33:20,347.152090,31.423035,41.076640,26.931602,43.109122,43.439556,1480.716492,70.010762,214812.836175,29.924235,43.256122,245.432231,1667.720491,27.195302,B2,Medium,2.611941,0.387368,27.654677
3,WM_28570,2019-06-26 03:53:20,24.471997,-99.000000,14.375078,66.513953,13.741253,15.577472,887.979475,41.445258,NaN,23.886434,13.501595,NaN,1329.744740,15.245757,BBB,Low,2.866805,0.450478,24.189426
4,WM_36934,2019-08-27 16:43:20,96.997026,33.281836,41.405192,1.843112,121.572907,43.934587,2053.916354,68.007787,16833.546517,35.906889,-99.000000,442.425744,691.408996,34.257024,A,Low,3.549672,0.368355,4.885440


In [7]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
wind_speed(m/s),27927.0,69.037071,76.275645,-4.962110e+02,20.883502,93.302129,95.268058,6.014557e+02
atmospheric_temperature(°C),24750.0,0.383727,44.278534,-9.900000e+01,7.948900,16.102410,23.687285,8.021744e+01
shaft_temperature(°C),28198.0,40.085387,27.204430,-9.900000e+01,41.633238,43.686082,45.673685,1.698205e+02
blades_angle(°),27984.0,-9.654038,47.918161,-1.462595e+02,-1.197652,-0.495608,5.495030,1.659321e+02
gearbox_temperature(°C),28199.0,41.027755,43.663605,-2.449741e+02,40.557952,43.221735,45.879425,9.990000e+02
engine_temperature(°C),28188.0,42.614239,6.124546,3.167151e+00,41.911365,43.525297,45.174246,5.000000e+01
motor_torque(N-m),28176.0,1710.819803,827.205537,5.000000e+02,870.340239,2031.849540,2462.585729,3.000000e+03
generator_temperature(°C),28188.0,65.027857,19.816499,3.389378e+01,41.198509,70.729533,78.945849,1.000000e+02
atmospheric_pressure(Pascal),25493.0,53185.064883,187503.615642,-1.188624e+06,16794.921490,18191.125867,118113.289822,1.272552e+06
area_temperature(°C),28200.0,32.735091,7.703391,-3.000000e+01,27.311644,32.605195,38.232387,5.500000e+01


In [8]:
train.dtypes

tracking_id                        object
datetime                           object
wind_speed(m/s)                   float64
atmospheric_temperature(°C)       float64
shaft_temperature(°C)             float64
blades_angle(°)                   float64
gearbox_temperature(°C)           float64
engine_temperature(°C)            float64
motor_torque(N-m)                 float64
generator_temperature(°C)         float64
atmospheric_pressure(Pascal)      float64
area_temperature(°C)              float64
windmill_body_temperature(°C)     float64
wind_direction(°)                 float64
resistance(ohm)                   float64
rotor_torque(N-m)                 float64
turbine_status                     object
cloud_level                        object
blade_length(m)                   float64
blade_breadth(m)                  float64
windmill_height(m)                float64
windmill_generated_power(kW/h)    float64
dtype: object

We can see that there are 18 continuous variables and 2 categorical variables (ignoring *tracking_id* and *datetime* as those donot seem important)

In [9]:
train.isna().sum()

tracking_id                          0
datetime                             0
wind_speed(m/s)                    273
atmospheric_temperature(°C)       3450
shaft_temperature(°C)                2
blades_angle(°)                    216
gearbox_temperature(°C)              1
engine_temperature(°C)              12
motor_torque(N-m)                   24
generator_temperature(°C)           12
atmospheric_pressure(Pascal)      2707
area_temperature(°C)                 0
windmill_body_temperature(°C)     2363
wind_direction(°)                 5103
resistance(ohm)                      1
rotor_torque(N-m)                  572
turbine_status                    1759
cloud_level                        276
blade_length(m)                   5093
blade_breadth(m)                     0
windmill_height(m)                 543
windmill_generated_power(kW/h)     207
dtype: int64

### Splitting features and target

In [10]:
numerical_feaures = ['wind_speed(m/s)', 'atmospheric_temperature(°C)', 'shaft_temperature(°C)', 'blades_angle(°)',
                     'gearbox_temperature(°C)', 'engine_temperature(°C)', 'motor_torque(N-m)', 'generator_temperature(°C)',
                     'atmospheric_pressure(Pascal)', 'area_temperature(°C)', 'windmill_body_temperature(°C)', 'wind_direction(°)',
                     'resistance(ohm)', 'rotor_torque(N-m)', 'blade_length(m)', 'blade_breadth(m)', 'windmill_height(m)']
categorical_features = ['turbine_status', 'cloud_level']

In [11]:
X = train[numerical_feaures+categorical_features].copy()
y = train['windmill_generated_power(kW/h)'].copy()

To handle the missing values, I am going to use the **mean** for numerical variables and **Missing** for categorical variables.

In [12]:
for feature in numerical_feaures:
  X[feature].fillna(X[feature].mean(), inplace=True)

In [13]:
X[categorical_features].fillna("Missing", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


I was thinking if I should remove the missing values from the target variable, but now I think I am going to fill them with mean values.

In [14]:
y.fillna(y.mean(), inplace=True)

### Converting some features

I think I should convert the **Celsius** values into **Kelvin**.

In [15]:
def C_to_K(row):
  return row+273.15

In [16]:
celsius_features = ['atmospheric_temperature(°C)', 'shaft_temperature(°C)', 'gearbox_temperature(°C)', 'engine_temperature(°C)',
                    'generator_temperature(°C)', 'area_temperature(°C)', 'windmill_body_temperature(°C)']
for feature in celsius_features:
  X[feature] = X[feature].apply(C_to_K)

Also **Degree** to **Radian**.

In [17]:
def Deg_to_Rad(row):
  return row*0.01745 # pi/180 yeilds 0.01745

In [18]:
for feature in ['blades_angle(°)', 'wind_direction(°)']:
  X[feature] = X[feature].apply(Deg_to_Rad)

Lets see what we have after these conversions:

In [19]:
X.head()

,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),turbine_status,cloud_level
0,94.820023,174.150000,314.873019,-0.015765,355.560573,315.673015,2563.124522,349.815560,103402.961872,300.047875,293.949761,4.185145,2730.310605,42.084666,2.217542,0.314065,24.281689,BA,Medium
1,241.832734,300.914785,174.150000,-1.727550,317.254919,319.408870,2372.384119,351.279803,17030.904078,312.951469,293.949761,5.897135,1780.207200,107.888643,4.210346,0.448494,27.262139,A2,Medium
2,95.484724,273.533727,315.005473,0.220791,315.472098,316.028552,1657.169646,340.804469,16125.927107,309.266065,318.183197,3.975988,1666.049900,-42.931459,2.719475,0.302321,27.366127,ABC,Medium
3,238.819424,174.150000,318.593914,0.263762,317.909643,320.432101,2888.134079,368.539974,18689.732336,319.170045,317.977154,8.586823,1964.502895,42.744596,4.857385,0.367140,24.287767,ABC,NaN
4,10.722890,273.533727,315.131183,0.029939,255.533541,316.619852,781.695419,310.573065,114468.169007,307.722941,174.150000,4.524342,1177.516152,13.387289,2.254034,0.453374,27.971650,AAA,Medium


Don't mind the column names!

### Splitting data Train and Test data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Scaling the data

### Converting Categorical Feature values into Numeric form

In [21]:
# convert categorical features to numerical value for modelling
def categorify(df, cat_vars):
  categories = {}
  for cat in cat_vars:
    df[cat] = df[cat].astype("category").cat.as_ordered()
    categories[cat] = df[cat].cat.categories
  return categories

# apply these values to test data
def apply_test(test, categories):
  for cat, index in categories.items():
    test[cat] = pd.Categorical(test[cat], categories=categories[cat], ordered=True)

In [22]:
categories = categorify(X_train, categorical_features)
apply_test(X_test, categories)

for cat in categorical_features:
  X_train[cat] = X_train[cat].cat.codes+1
  X_test[cat] = X_test[cat].cat.codes+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

## Model

In [23]:
# get embedding size for each categorical variable
def get_emb_sz(cat_col, categories_dict):
  num_classes = len(categories_dict[cat_col])
  return int(min(600, round(1.6*num_classes**0.56)))

This function was created following [Jeremy Howard’s ‘Introduction to Machine Learning for Coders’](https://course18.fast.ai/ml) course.

In [24]:
def define_model(categorical_features, categories_dict, numerical_feaures, layers):
  inputs = []
  embeddings = []
  emb_dict ={}
  
  # create embedding layer for each categorical variables
  for cat in categorical_features:
    emb_dict[cat] = Input(shape=(1,))
    emb_sz = get_emb_sz(cat, categories_dict)
    vocab = len(categories_dict[cat])+1
    embedding = Embedding(vocab, emb_sz, input_length=1)(emb_dict[cat])
    embedding = Reshape(target_shape=(emb_sz,))(embedding)
    inputs.append(emb_dict[cat])
    embeddings.append(embedding)
  
  # concat numerical variables with embedded variables
  cont_input = Input(shape=(len(numerical_feaures),))
  embedding = BatchNormalization()(cont_input)
  inputs.append(cont_input)
  embeddings.append(embedding)
  x = Concatenate()(embeddings)
  
  # add user-defined fully-connected layers separated with batchnorm and dropout layers
  for i in range(len(layers)):
    if i == 0:
      x = Dense(layers[i], activation="relu")(x)
    else:
      x = BatchNormalization()(x)
      x = Dropout(0.5)(x)
      x = Dense(layers[i], activation="relu")(x)
  output = Dense(1)(x)
  model = Model(inputs, output)
  return model

In [25]:
layers = [200, 100]
model = define_model(categorical_features, categories, numerical_feaures, layers)
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=["mean_squared_error", "cosine_similarity"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 7)         105         input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 3)         12          input_2[0][0]                    
______________________________________________________________________________________________

In [26]:
# process x_train input to fit model
input_list = []
for cat in categorical_features:
  input_list.append(X_train[cat].values)
input_list.append(X_train[numerical_feaures].values)

# modify x_test input to fit model
test_list = []
for cat in categorical_features:
  test_list.append(X_test[cat].values)
test_list.append(X_test[numerical_feaures].values)

In [27]:
model.fit(input_list, y_train, epochs=10)

Epoch 1/10
705/705 [==============================] - 3s 3ms/step - loss: 19.2851 - mean_squared_error: 19.2851 - cosine_similarity: 0.7369
Epoch 2/10
705/705 [==============================] - 2s 3ms/step - loss: 5.2472 - mean_squared_error: 5.2472 - cosine_similarity: 0.9988
Epoch 3/10
705/705 [==============================] - 2s 3ms/step - loss: 4.3791 - mean_squared_error: 4.3791 - cosine_similarity: 0.9996
Epoch 4/10
705/705 [==============================] - 2s 3ms/step - loss: 3.9307 - mean_squared_error: 3.9307 - cosine_similarity: 1.0000
Epoch 5/10
705/705 [==============================] - 2s 3ms/step - loss: 3.6344 - mean_squared_error: 3.6344 - cosine_similarity: 1.0000
Epoch 6/10
705/705 [==============================] - 2s 3ms/step - loss: 3.4520 - mean_squared_error: 3.4520 - cosine_similarity: 0.9999
Epoch 7/10
705/705 [==============================] - 2s 3ms/step - loss: 3.2935 - mean_squared_error: 3.2935 - cosine_similarity: 0.9999
Epoch 8/10
705/705 [============

In [28]:
y_pred = model.predict(test_list)

In [29]:
metrics.r2_score(y_test, y_pred)

0.7384828640906442

In [30]:
# def test_data_prediction(test, scaler, categories, model):
def test_data_prediction(test, categories, model):
  # selecting the features that i want
  X = test[numerical_feaures+categorical_features].copy()
  # fill the missing values
  for feature in numerical_feaures:
    X[feature].fillna(X[feature].mean(), inplace=True)
  X[categorical_features].fillna("Missing", inplace=True)
  # convert Celsius to Kelvin
  for feature in celsius_features:
    X[feature] = X[feature].apply(C_to_K)
  # convert Degree to Radian
  for feature in ['blades_angle(°)', 'wind_direction(°)']:
    X[feature] = X[feature].apply(Deg_to_Rad)
  # scale the data
  # X[numerical_feaures] = scaler.transform(X[numerical_feaures])
  # convert categorical feature values into numeric form
  apply_test(X, categories)
  for cat in categorical_features:
    X[cat] = X[cat].cat.codes+1
  # prepare the data for input
  test_list = []
  for cat in categorical_features:
    test_list.append(X[cat].values)
  test_list.append(X[numerical_feaures].values)

  return model.predict(test_list)

In [34]:
# predictions = test_data_prediction(test, scaler, categories, model)
predictions = test_data_prediction(test, categories, model)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [35]:
output_df = test[['tracking_id', 'datetime']]
output_df['windmill_generated_power(kW/h)'] = pd.DataFrame(data=predictions, columns=['windmill_generated_power(kW/h)'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
output_df.to_csv('predictions (#3).csv')